In [1]:
from azureml.core import Workspace, Experiment
import pandas as pd



ws = Workspace.from_config()

# ws = Workspace.get(name='quick-starts-ws-130632')

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-130632
Azure region: southcentralus
Subscription id: 2248b1c7-a065-4265-9476-038b58f0650e
Resource group: aml-quickstarts-130632


In [2]:
# !pip install --upgrade --force-reinstall -r https://aka.ms/automl_linux_requirements.txt

In [3]:
# !pip install --upgrade --force-reinstall -r https://automlcesdkdataresources.blob.core.windows.net/validated-requirements/1.19.0/validated_linux_requirements.txt

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster

cpu_cluster_name = 'cpucluster'

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.





try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)
### YOUR CODE HERE ###

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C" : choice(0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000),
                            "--max_iter": choice(50,100,150,200,300,400,500)})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval = 1,delay_evaluation=5)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./training",entry_script="train.py",compute_target=cpu_cluster_name)
### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =  HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    max_total_runs=100,
    max_concurrent_runs=4,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    estimator=est
)

### YOUR CODE HERE ###

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment=Experiment(ws,ws.name)
hyperdrive_run = experiment.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)
assert(hyperdrive_run.get_status() == "Completed")
### YOUR CODE HERE ###

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_632129ec-7909-4a29-91dc-8d662ad2390b
Web View: https://ml.azure.com/experiments/quick-starts-ws-130632/runs/HD_632129ec-7909-4a29-91dc-8d662ad2390b?wsid=/subscriptions/2248b1c7-a065-4265-9476-038b58f0650e/resourcegroups/aml-quickstarts-130632/workspaces/quick-starts-ws-130632

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-15T19:24:06.194072][API][INFO]Experiment created<END>\n"<START>[2020-12-15T19:24:07.1408559Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-12-15T19:24:09.184898][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"

Execution Summary
RunId: HD_632129ec-7909-4a29-91dc-8d662ad2390b
Web View: https://ml.azure.com/experiments/quick-starts-ws-130632/runs/HD_632129ec-7909-4a29-91dc-8d662ad2390b?wsid=/subscriptions/2248b1c7-a065-4265-9476-038b58f0650e/resourcegroups/aml-quickstarts-130632/workspaces/quick-starts-ws-130632



In [19]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength: ',parameter_values[1])
print('\n Max iterations: ',parameter_values[3])



### YOUR CODE HERE ###

Best Run Id:  HD_632129ec-7909-4a29-91dc-8d662ad2390b_15

 Accuracy: 0.910065756196257

 Regularization Strength:  0.001

 Max iterations:  400


In [18]:
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_8ed481997ee1a977588137534df7bf8aedda1b781c5e51893825d3bc37e9fd31_d.txt',
 'azureml-logs/65_job_prep-tvmps_8ed481997ee1a977588137534df7bf8aedda1b781c5e51893825d3bc37e9fd31_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_8ed481997ee1a977588137534df7bf8aedda1b781c5e51893825d3bc37e9fd31_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/104_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/engine_spans_l_05b358fa-6d00-4734-a02e-8b47fdf52f7f.jsonl',
 'logs/azureml/dataprep/python_span_l_05b358fa-6d00-4734-a02e-8b47fdf52f7f.jsonl',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log']

In [22]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
from azureml.core import Workspace, Dataset

# dataset = Dataset.get_by_name(ws, name='Bank-marketing')
# ds = dataset.to_pandas_dataframe()


data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(path=data_path)
ds = dataset.to_pandas_dataframe()

### YOUR CODE HERE ###

In [23]:
from training.train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

df = x.join(y)


train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
label = "y"

In [24]:
from azureml.core import Dataset
#Export df to csv

path = './outputs/'

train_data.to_csv(path+'train_data.csv')
test_data.to_csv(path+'test_data.csv')

#Import csv as dataset
datastore=ws.get_default_datastore()
datastore.upload(src_dir=path)

training_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('train_data.csv'))])
test_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('test_data.csv'))])

Uploading an estimated of 2 files
Target already exists. Skipping upload for test_data.csv
Target already exists. Skipping upload for train_data.csv
Uploaded 0 files


In [25]:
df.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [27]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=training_dataset,
    label_column_name=label,
    n_cross_validations=5,
    compute_target = cpu_cluster)

In [28]:
# Submit your automl run
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails
# experiment=Experiment(ws,ws.name)

exp = Experiment(workspace=ws, name="AutoML-project")
automl_run = exp.submit(config = automl_config)


RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)
assert(automl_run.get_status() == "Completed")


### YOUR CODE HERE ###

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
displayHTML("<a href={} target='_blank'>Azure Portal: {}</a>".format(automl_run.get_portal_url(), automl_run.id))

In [ ]:
# Retrieve and save your best automl model.
automl_best_run, automl_fitted_model = automl_run.get_output()

print("Fitted model: ", automl_fitted_model.steps[-1])
print("AutoML best run accuracy: ", automl_best_run.get_metrics(name="accuracy"))
print("AutoML run Summary: ", automl_run.summary())
### YOUR CODE HERE ###

In [ ]:
#Save the model
automl_best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')